In [12]:
import os
import json
import time
import requests
import datetime
import dateutil
import pandas as pd
from dateutil.relativedelta import relativedelta

In [27]:
import os
import sys
import pandas as pd
import numpy as np
import json
import time
import datetime
import requests
import pickle
import subprocess
from bs4 import BeautifulSoup

In [13]:
from pynytimes import NYTAPI

In [20]:
nyt = NYTAPI("bDNNuZv46VmAfG12ESSXn83y2Ia3GvJP")

In [23]:
top_stories = nyt.top_stories()
top_science_stories = nyt.top_stories(section = "science")
print(top_science_stories)

[{'section': 'science', 'subsection': '', 'title': 'He Was a Stick, She Was a Leaf; Together They Made History', 'abstract': 'A surprise clutch of eggs has solved a century-old leaf insect mystery.', 'url': 'https://www.nytimes.com/2020/12/01/science/leaf-stick-insects-phyllium-asekiense.html', 'uri': 'nyt://article/7503d272-f1b8-5be0-b68a-99614b7a7291', 'byline': 'By Sabrina Imbler', 'item_type': 'Article', 'updated_date': '2020-12-01T02:30:12-05:00', 'created_date': '2020-12-01T02:30:12-05:00', 'published_date': '2020-12-01T02:30:12-05:00', 'material_type_facet': '', 'kicker': '', 'des_facet': ['Reproduction (Biological)', 'Insects', 'your-feed-science', 'your-feed-animals', 'Biology and Biochemistry', 'Evolution (Biology)'], 'org_facet': ['Insectarium', 'ZooKeys (Journal)'], 'per_facet': ['Le Tirant, Stephane', 'Cumming, Royce', 'Brock, Paul'], 'geo_facet': ['Papua New Guinea', 'Montreal (Quebec)'], 'multimedia': [{'url': 'https://static01.nyt.com/images/2020/12/01/science/00SCI-STI

In [25]:
most_viewed = nyt.most_viewed()

# Get most viewed articles of last 7 or 30 days
most_viewed = nyt.most_viewed(days = 7)
most_viewed = nyt.most_viewed(days = 30)
print(most_viewed)

[{'uri': 'nyt://article/e74bf800-4d6f-5d09-8404-0d5ac060263b', 'url': 'https://www.nytimes.com/2020/11/07/us/politics/biden-election.html', 'id': 100000007435608, 'asset_id': 100000007435608, 'source': 'New York Times', 'published_date': '2020-11-07', 'updated': '2020-11-18 09:50:53', 'section': 'U.S.', 'subsection': 'Politics', 'nytdsection': 'u.s.', 'adx_keywords': 'Presidential Election of 2020;United States Politics and Government;internal-election-open;Biden, Joseph R Jr;Trump, Donald J;Harris, Kamala D;Democratic Party', 'column': None, 'byline': 'By Jonathan Martin and Alexander Burns', 'type': 'Article', 'title': 'Biden Wins Presidency, Ending Four Tumultuous Years Under Trump', 'abstract': 'Joseph R. Biden Jr. achieved victory offering a message of healing and unity. He will return to Washington facing a daunting set of crises.', 'des_facet': ['Presidential Election of 2020', 'United States Politics and Government', 'internal-election-open'], 'org_facet': ['Democratic Party'],

In [29]:
def get_all_articles(articles):
    years_num = list(np.arange(1981,2019))
    years = []

    for year in years_num:
        years.append(str(year))

    months_num = list(np.arange(1,13))
    months = []
    daysIn = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

    for month in months_num:
        months.append(str(month))
    for year in years:
        articles[year] = {}
        for month in months:
            if year == '2019' and month == '5':
                break
            articles[year][month] = {}
            print('Getting {}/{} articles'.format(year,month))
            #url = 'https://api.nytimes.com/svc/search/v2/articlesearch.json'
            url = 'https://api.nytimes.com/svc/archive/v1/{}/{}.json'.format(year,month)

            # begin = ""
            # if len(month) == 1:
            #     month = "0" + month
            # begin += year + month + "01"
            # endDay = str(daysIn[int(month) - 1])
            # end = year + month + endDay




            #params = {'begin_date': begin, 'end_date': end ,'api-key': api_key}
            params = {'api-key': api_key}
            response = requests.get(url, params=params)
            print(response)

            while response.status_code != 200:
                print('trying again...')
                time.sleep(3)
                response = requests.get(url, params=params)
                print('status code: {}'.format(response.status_code))

            articlesjson = json.loads(response.text)
            print(articlesjson)
            return
            docs = articlesjson['response']['docs']

            articles[year][month] = len(docs)

        print('Dumping year {}.'.format(year))

        with open('archive_export.json', 'w') as fp:
            json.dump(articles, fp)

    print('Exporting articles 1981_1-2018_2 to json')
    with open('archive_export.json', 'w') as fp:
        json.dump(articles, fp)


def get_metadata():
    begin_dates = [20000101, 20010101, 20020101]
    end_dates = [20001231, 20011231, 20021231]
    begin_date = 19930101
    end_date = 19931231
    index = 1
    articles = {}
    exportno = 5
    page = 1

    while page < 101 and exportno < 50 and index <= 3:
        response = None
        if len(articles)>=1000:
            print('Exporting results till page {} to json {}'.format(page-1,exportno))
            with open('metadata_export{}.json'.format(exportno), 'w') as fp:
                json.dump(articles, fp)
            articles = {}
            exportno += 1


#         print('Getting articles from page {}'.format(page))

        #Foreign, Business
        url = 'https://api.nytimes.com/svc/search/v2/articlesearch.json'
        news = "section_name:(\"Your Money\" \"Job Market\" \"Business\" \"World\" \"Business Day\" \"Technology\") AND document_type:(\"article\") AND type_of_material:(\"news\")"

        tokens = "Job Market"
        params = {'api-key': api_key,
                  'begin_date': begin_date,
                  'end_date': end_date,
                  'fq': news,
                  'page': page}

        while response == None:
            try:
                response = requests.get(url, params=params)
                break
            except requests.exceptions.ConnectionError:
                time.sleep(5)
                continue


#         print('status code: {}'.format(response.status_code))

        time.sleep(0.6)
        temp = False
        while temp or response.status_code != 200:
            print('trying again...')
            time.sleep(3)
            try:
                response = requests.get(url, params=params)
                if response.status_code == 200:
                    break
            except requests.exceptions.ConnectionError:
                temp = True
                continue

        articlesjson = json.loads(response.text)

        docs = articlesjson['response']['docs']

        try:
            for i in range(0,len(docs)):
                item = docs[i]

                # Get URL
                articles[item['web_url']] = {}

                # Get Name of Writer
                try:
                    articles[item['web_url']]['writer_name'] = item['byline']['person'][0]['firstname']+' '+item['byline']['person'][0]['lastname']
                except:
                    articles[item['web_url']]['writer_name'] = None

                # Get Publication Date
                try:
                    articles[item['web_url']]['pub_date'] = item['pub_date'][:10]
                    tempdate = ''.join((item['pub_date'][:10]).split('-'))
                except:
                    articles[item['web_url']]['pub_date'] = None

                # Get Snippet
                try:
                    articles[item['web_url']]['snippet'] = item['snippet']
                except:
                    articles[item['web_url']]['snippet'] = None

                # Get Word Count
                try:
                    articles[item['web_url']]['word_count'] = item['word_count']
                except:
                    articles[item['web_url']]['word_count'] = None

                 # Get Score
                try:
                    articles[item['web_url']]['score'] = item['score']
                except:
                    articles[item['web_url']]['score'] = None

                # Get Source
                try:
                    articles[item['web_url']]['source'] = item['source']
                except:
                    articles[item['web_url']]['source'] = None

                # Get Section Name
                try:
                    articles[item['web_url']]['section_name'] = item['section_name']
                except:
                    articles[item['web_url']]['section_name'] = None

                # Get Type of Material
                try:
                    articles[item['web_url']]['type_of_material'] = item['type_of_material']
                except:
                    articles[item['web_url']]['type_of_material'] = None

                # Get Document Type
                try:
                    articles[item['web_url']]['document_type'] = item['document_type']
                except:
                    articles[item['web_url']]['document_type'] = None

                # Get Main / Web Headline
                try:
                    articles[item['web_url']]['main_headline'] = item['headline']['main']
                except:
                    articles[item['web_url']]['main_headline'] = None

                # Get Print Headline
                try:
                    articles[item['web_url']]['print_headline'] = item['headline']['print_headline']
                except:
                    articles[item['web_url']]['print_headline'] = None

            page += 1

            if page%10==0 and page>0:
                print('Get articles page {} success'.format(page))

        except Exception as e:
            print('ERROR:, {}'.format(e))
            return {}

        # if page == 200:
        #     end_date = end_dates[index]
        #     begin_date = begin_dates[index]
        #     index += 1
        #     page = 0
        #     break

    print('Exporting remainder including page {} to json {}'.format(page,exportno))
    with open('metadata_export{}.json'.format(exportno), 'w') as fp:
        json.dump(articles, fp)
def get_bodytext():
    #arr = ["metadata_1981_2", "metadata_1982_1", "metadata_1982_2", "metadata_1982_3", "metadata_1983_1", "metadata_1983_2", "metadata_1983_3", "metadata_1984_1", "metadata_1984_2", "metadata_1984_3", "metadata_1985_1", "metadata_1985_2", "metadata_1986_1", "metadata_1986_2", "metadata_1986_3", "metadata_1987_1", "metadata_1987_2", "metadata_1987_3", "metadata_1988_1", "metadata_1988_2", "metadata_1988_3", "metadata_1989_1", "metadata_1989_2", "metadata_1989_3", "metadata_1990_1", "metadata_1990_2", "metadata_1990_3", "metadata_1991_1", "metadata_1991_2", "metadata_1992_1", "metadata_1992_2", "metadata_1993_1", "metadata_1993_2", "metadata_1994_1", "metadata_1994_2", "metadata_1995_1", "metadata_1995_2", "metadata_1996_1", "metadata_1996_2", "metadata_1997_1", "metadata_1997_2", "metadata_1998_1", "metadata_1998_2", "metadata_1999_1", "metadata_1999_2", "metadata_2000_1", "metadata_2000_2", "metadata_2001_1", "metadata_2001_2", "metadata_2002_1", "metadata_2002_2"]
    arr = ["metadata_2017_1", "metadata_2017_2", "metadata_2018_1", "metadata_2018_2"]
    i = 0
    for file in arr:
        name = "Temp/"
        name += file
        name += ".json"
        with open(name) as fp:
            articles = json.load(fp)

        for url in articles.keys():
            print(i)
            i += 1
#             print(url)
            page = None
            while page == None:
                try:
                    #page = requests.get(url, timeout=9)
                    page = requests.get(url)
                    break
                except requests.exceptions.ConnectionError:
                    time.sleep(5)
                    continue
                except requests.exceptions.Timeout:
                    print("not good")
                    time.sleep(45)
                    continue

#             time.sleep(0.2)
            soup = BeautifulSoup(page.text, 'lxml')
            text = soup.findAll(attrs={'class':'story-body-text story-content'})
            if text == []:
                text = soup.findAll(attrs={'class':'story-body-text'})
            if text == []:
                text = soup.findAll(attrs={'itemprop':'articleBody'})
            if text == []:
                text = soup.findAll(attrs={'itemprop':'reviewBody'})
            body_text = ''
            for paragraph in text:
                body_text += (' **********'+paragraph.get_text())
            articles[url]['body_text'] = body_text



#             time.sleep(0.6)
        # Write to .json
        print('{}: exporting to bodytext_export{}.json'.format(datetime.datetime.now(),i))
        output = "FullOutput/"
        output += file[9:]
        output += "_fulltext"
        output += ".json"
        with open(output, 'w') as fp:
            json.dump(articles, fp)

    print("end")
    print(i)
    print("articles")

if 'darwin' in sys.platform:
    print('Running \'caffeinate\' on MacOSX to prevent the system from sleeping')
    subprocess.Popen('caffeinate')
#get_metadata()
get_bodytext()
#get_all_articles(articles)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1848
1849
1850
1851
1852
1853
1854
1855
1856
1857
1858
1859
1860
1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2020-12-01 17:03:56.526664: exporting to bodytext_export2000.json
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034

3461
3462
3463
3464
3465
3466
3467
3468
3469
3470
3471
3472
3473
3474
3475
3476
3477
3478
3479
3480
3481
3482
3483
3484
3485
3486
3487
3488
3489
3490
3491
3492
3493
3494
3495
3496
3497
3498
3499
3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
